## notebook for testing entropy viscosity refactoring 

In [ ]:
import  math
import  numpy as  np
from proteus.iproteus import *
Profiling.logLevel=5
Profiling.verbose=True
import proteus as pr
import cev_utils

In [ ]:
reload(cev_utils)

## do enough to setup a numerical solution with necessary infrastructure 

### define the problem

In [ ]:
class LA(TransportCoefficients.TC_base):
    """
    The coefficients of the linear advection-diffusion equation
    """
    def __init__(self):
        TransportCoefficients.TC_base.__init__(self,
                                               nc=1, #number of components
                                               variableNames=['u'],
                                               mass      = {0:{0:'linear'}},
                                               advection = {0:{0:'nonlinear'}})#,
                                               #diffusion = {0:{0:{0:'constant'}}},
                                               #potential = {0:{0:'u'}})

    def evaluate(self,t,c):
        c[('m',0)][:]         = c[('u',0)]
        c[('dm',0,0)][:]      = 1.0
        c[('f',0)][...,0]     = -2.0*math.pi*c['x'][...,1]*c[('u',0)]
        c[('f',0)][...,1]     =  2.0*math.pi*c['x'][...,0]*c[('u',0)]
        c[('df',0,0)][...,0]  = -2.0*math.pi*c['x'][...,1]
        c[('df',0,0)][...,1]  =  2.0*math.pi*c['x'][...,0]


### physics

In [ ]:
from proteus import default_p as p
#physics
p.name = "la"
p.nd = 2; #Two dimensions
box=Domain.RectangularDomain(L=(2.0,2.0),
                             x=(-1.0,-1.0),
                             name="la");
box.writePoly("la")
p.domain=Domain.PlanarStraightLineGraphDomain(fileprefix="la")

p.T=0.005

p.coefficients=LA()

def getDBC(x,flag):
    return None
#    if flag in ['left','right','top','bottom']:
#        return lambda x,t: 0.0

def getAFBC(x,flag):
    if flag in ['left','right','top','bottom']:
        return lambda x,t: 0.0

p.dirichletConditions = {0:getDBC}
p.advectiveFluxBoundaryConditions = {0:getAFBC}

class IC_3B:
    def __init__(self,
                 xd=(0,0.5),
                 xc=(0,-0.5),
                 xh=(-0.5,0),
                 r0=0.3):
        self.xd=np.array(xd+(0.0,))
        self.xc=np.array(xc+(0.0,))
        self.xh=np.array(xh+(0.0,))
        self.r0=r0
    def g(self, r):
        from math import cos, pi
        return (1.+cos(pi*min(r/self.r0,1.)))/4.0
    def uOfXT(self,x, t):
        from math import fabs
        import numpy as  np
        from numpy.linalg import norm
        x=np.array(x)
        rd = norm(x-self.xd)
        rc = norm(x-self.xc)
        rh = norm(x-self.xh)
        if ( rd <= self.r0 and
             (fabs(x[0]) >= 0.05 or x[1] >= 0.7)):
            return 1.0
        elif rc <= self.r0:
            return 1. - rc/self.r0
        elif rh <= self.r0:
            return self.g(rh)
        else:
            return 0.0

p.initialConditions  = {0:IC_3B()}



### numerics

In [ ]:
from proteus import default_n as n

timeOrder=1; spaceOrder=1
he = 0.05
n.triangleOptions="VApq33Dena%8.8f" % (he**2/2.0,)
n.timeIntegration = pr.TimeIntegration.LinearSSPRKintegration
n.nStagesTime = 1
n.stepController = pr.StepControl.Min_dt_RKcontroller
n.runCFL=0.33

n.femSpaces = {0:pr.FemTools.C0_AffineLinearOnSimplexWithNodalBasis}
n.elementQuadrature = pr.Quadrature.SimplexLobattoQuadrature(p.nd,1)
n.elementBoundaryQuadrature = pr.Quadrature.SimplexLobattoQuadrature(p.nd-1,1)
n.subgridError = pr.SubgridError.AdvectionLag_ASGS(p.coefficients,
                                                   p.nd,
                                                   lag=False)

n.shockCapturing = pr.ShockCapturing.ResGradQuadDelayLag_SC(p.coefficients,
                                                            p.nd,
                                                            shockCapturingFactor=0.0,
                                                            nStepsToDelay=1,
                                                            lag=False)
n.numericalFluxType = pr.NumericalFlux.Advection_DiagonalUpwind_exterior
n.nDTout=1
n.tnList=[float(i)*p.T/float(n.nDTout) for i in range(n.nDTout+1)]
n.matrix = pr.LinearAlgebraTools.SparseMatrix
n.multilevelLinearSolver = pr.LinearSolvers.LU
n.l_atol_res = 0.0
n.linTolFac = 0.001
n.tolFac = 0.0
n.nl_atol_res = 1.0e-4
n.maxNonlinearIts =1
n.maxLineSearches =0
n.parallelPartitioningType = pr.MeshTools.MeshParallelPartitioningTypes.node
n.nLayersOfOverlapForParallel = 0
n.periodicDirichletConditions = None


### operator splitting

In [ ]:
from proteus import default_s,default_so
so = default_so
p.name += "k{0}_p{1}_h{2}".format(timeOrder,spaceOrder,he)

so.name = p.name
so.sList=[default_s]
so.tnList = n.tnList
from proteus.Archiver import ArchiveFlags
so.archiveFlag = ArchiveFlags.EVERY_USER_STEP
so.systemStepControllerType = SplitOperator.Sequential_MinAdaptiveModelStep


## initialize the numerical solution and grab a pointer to the model

In [ ]:
ns = NumericalSolution.NS_base(so,[p],[n],so.sList,opts)
model = ns.modelList[0].levelModelList[-1]

### run the model for a few steps to make sure all the necessary data structures are available

In [ ]:
failed = ns.calculateSolution('la_k{0}_p{1}_h{2}'.format(timeOrder,spaceOrder,he))
assert(not failed)


### Test b_ij generation

In [ ]:
model.use_bij

What about just using the mesh information

In [ ]:
b_copy = model.b.copy(); b_copy.fill(0.)
b_copy = cev_utils.build_graph_laplacian_element(np.sum(model.q['dV'],1),b_copy,model.nDOF_trial_element[0])
assert np.allclose(model.b,b_copy)
